# Practica 1

In [12]:
## Intro

In [13]:
!cd ~/Documentos/GitHub/FI/Practica1/
!ls -l

total 4
-rw-rw-r-- 1 patricia patricia 589 oct 26 14:38 Untitled.ipynb


In [15]:
!head errors.txt

head: no se puede abrir 'errors.txt' para lectura: No existe el archivo o el directorio
